## Create a template for event remapping.
This notebook works through the process analyzing the contents of the event files in
a BIDS dataset and creating a template for remapping event codes.

The example used in this notebook is reduced version of an auditory attention shift
dataset which is available at
[https://github.com/hed-standard/hed-examples/data/eeg_ds0028932](https://github.com/hed-standard/hed-examples/data/eeg_ds0028932).
To run this notebook, you will need download this dataset and set the `bids_root_path`
variable to the local path.

The goal is to remap each combination in the key columns [`value`, `trial_type`]
into a new combination in the target columns [`event_type`, `task_role`, `shift_cond`]
in order to have a more orthogonal event design and make the meanings of contents
of the event file more clear.



In [1]:
# bids_root_path = "G:/AttentionShift/AttentionShiftExperiments"
bids_root_path = "D:/eeg_ds002893s"
key_columns = ["value", "trial_type"]
target_columns = ["event_type", "task_role", "shift_cond"]


The next example traverses the directory tree to produce a list of event file paths.

In [2]:
from hed.tools import get_file_list
event_files = get_file_list(bids_root_path, types=[".tsv"], suffix="_events")
print(f"Bids dataset {bids_root_path} has {len(event_files)} event files")

Bids dataset D:/eeg_ds002893s has 6 event files


### Find the unique value combinations in a list of event columns

The tools traverse through the BIDS data set and gathers the unique combinations of
values in the key columns [`value`, `trial_type`].

If you just want a summary of the unique values in individual columns of all
event files, use the `ColumnDict` class as illustrated in notebook
`summarize_events.ipynb`.

use the `KeyTemplate` class to find the unique value combinations of a list of columns.
In the following example, We use the same list of files as in the `summarize_events.ipynb`
notebook, but count the number of times each unique
combinations of values that appear in the `value` and `trial_type` columns.

In [3]:
from hed.tools import KeyTemplate
template = KeyTemplate(key_columns)
for file in event_files:
    template.update(file)
template.resort()
template.print()

Counts for key [['value', 'trial_type']]:
[11, 1]	228
[12, 1]	228
[13, 1]	456
[14, 1]	456
[15, 1]	1822
[16, 1]	1820
[21, 2]	252
[22, 2]	252
[23, 2]	505
[24, 2]	504
[25, 2]	2010
[26, 2]	2012
[28, 2]	2
[31, 3]	720
[32, 3]	719
[37, 3]	960
[38, 3]	960
[39, 3]	480
[202, 1]	13
[202, 2]	17
[202, 3]	42
[212, 2]	3
[310, 3]	480
[311, 3]	3838
[312, 3]	3832
[313, 3]	1920
[314, 3]	1915
[1201, 1]	433
[2201, 2]	502
[3201, 3]	1846


### Make a template for remapping existing event values

If you decide to recode your events, you will need to create a template for doing the
remapping. The following code creates a data frame template and writes it as a `.tsv`.

In this case we decided to recode the events files so that each unique combination of
the key columns (e.g., [`value`, `trial_type`] in our example) will be translated
into a specific combination of the new columns [`event_type`, `task_role`, `shift_cond`].

The final template will consist of the 5 columns: [`value`, `trial_type`, `event_type`,
`task_role`, `shift_cond`] representing this association. The template is output.

In [4]:
import os
df = template.make_template(additional_cols=target_columns)
template_file = os.path.join(bids_root_path, "event_template.tsv")
df.to_csv(template_file, sep='\t', index=False)

PermissionError: [Errno 13] Permission denied: 'D:/eeg_ds002893s\\event_template.tsv'

### Fill in the template the mapping

To do remapping, you will need to edit the `event_template.tsv` spreadsheet
created in the previous step to indicate how each combination of [`value`, `trial_type`]
is mapped into a combination of [`event_type`, `task_role`, `shift_cond`].

Once you have filled in the mapping, you can use the `remap_events.ipynb` notebook
to remap all the event files in your BIDS dataset.